[View in Colaboratory](https://colab.research.google.com/github/ameasure/colab_tutorials/blob/master/FastAI.ipynb)

Download the necessities, specifically:

1.   pytorch (a library for building neural networks)
2.   fastai (a library on top of pytorch)
3.   spacy english language model
4.   msha.xlsx data
5.   xlrd (a library to read excel files)

In [11]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai
!python -m spacy download en
!wget 'https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx'
!pip install xlrd

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 574.9MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62398000 @  0x7efd4d0c51c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')


Redirecting output to ‘wget-log.2’.


In [13]:
import pandas as pd

df = pd.read_excel('msha.xlsx')
df['ACCIDENT_YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)
df['ACCIDENT_YEAR'].value_counts()
df_train = df[df['ACCIDENT_YEAR'].isin([2010, 2011])].copy()
df_valid = df[df['ACCIDENT_YEAR'] == 2012].copy()
print('training rows:', len(df_train))
print('validation rows:', len(df_valid))

training rows: 18681
validation rows: 9032


In [0]:
from sklearn.preprocessing import LabelEncoder

labeler = LabelEncoder().fit(df['INJ_BODY_PART'])
labeler.classes_
df_train['LABEL'] = labeler.transform(df_train['INJ_BODY_PART'])
df_valid['LABEL'] = labeler.transform(df_valid['INJ_BODY_PART'])

In [35]:
df_train[['LABEL', 'NARRATIVE']].to_csv('train.csv', header=False)
df_valid[['LABEL', 'NARRATIVE']].to_csv('valid.csv', header=False)
n_labels = len(labeler.classes_)
print(n_labels)

46


In [0]:
from fastai.text.data import TextDataset
from fastai.text.data import text_data_from_csv
from fastai.text.data import lm_data

In [38]:
train_ds = text_data_from_csv('.')

Tokenizing train.
Numericalizing train.
Tokenizing valid.
Numericalizing valid.


In [0]:
data_lm = text_data_from_csv(path='.', train='train', data_func=lm_data)

In [0]:
import os

os.mkdir('models')

In [0]:
from fastai.core import download_url

download_url('http://files.fast.ai/models/wt103_v1/lstm_wt103.pth', 'models/lstm_wt103.pth')
download_url('http://files.fast.ai/models/wt103_v1/itos_wt103.pkl', 'models/itos_wt103.pkl')

In [0]:
from fastai.docs import download_wt103_model

download_wt103_model()

In [0]:
from fastai.text.learner import RNNLearner

learn = RNNLearner.language_model(data_lm, pretrained_fnames=['lstm_wt103', 'itos_wt103'], drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch  train loss  valid loss  accuracy


In [0]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)

In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = RNNLearner.classifier(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))